## Library

In [ ]:
import os
import time
import logging
import warnings
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from pydantic import BaseModel, Field
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
)
from sentence_transformers import SentenceTransformer
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.callbacks import get_openai_callback
from langchain_huggingface import HuggingFaceEmbeddings

from dotenv import load_dotenv
load_dotenv()


warnings.filterwarnings("ignore")

In [ ]:
seed=1
retrieve_k = 5

## Data

In [2]:
train = pd.read_csv(f'../seed{seed}/seed{seed}_train.csv')
validation = pd.read_csv(f'../seed{seed}/seed{seed}_validation.csv')
test = pd.read_csv(f'../seed{seed}/seed{seed}_test.csv')

## Vectorstore

In [ ]:
device = torch.device("cpu")

class KUREEmbedding:
    def __init__(self, model_name="nlpai-lab/KURE-v1"):
        self.model = SentenceTransformer(model_name, trust_remote_code=True).to(device)

    def embed_documents(self, texts):
        embeddings = self.model.encode(texts, convert_to_numpy=True)
        return embeddings

    def embed_query(self, text):
        return self.embed_documents([text])[0]

class KoE5Embedding(KUREEmbedding):
    def __init__(self, model_name="nlpai-lab/KoE5"):
        super().__init__(model_name)

In [ ]:
vectorstore_path = f"../seed{seed}/faiss_index_seed{seed}_koe5"
if os.path.exists(vectorstore_path):
    embeddings = KoE5Embedding()

    vectorstore = FAISS.load_local(
        vectorstore_path,
        embeddings.embed_query,
        allow_dangerous_deserialization=True,
    )

    retriever = vectorstore.as_retriever(search_kwargs={"k": retrieve_k})

## Classifier

In [13]:
class SentenceClassification(BaseModel):
    prediction: str = Field(description="Classification of the sentence: True, False")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
structured_llm_grader = llm.with_structured_output(SentenceClassification)

## Prompt & Chain

In [ ]:
multiquery_prompt = ChatPromptTemplate.from_messages([
        ("system", "Generate two alternative versions of the given sentence in Korean, each within 50 characters."),
        ("human", "Original sentence: {question}")
    ])
 
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm=llm,
    prompt=multiquery_prompt
)

classification_prompt = ChatPromptTemplate.from_messages(
    [("system", "Classify the sentence into one of : True , False. "
                "Judge as True if it is unethical.\n"
                "Return in JSON: {{\"prediction\": \"True\" or \"False\"}}"),
     ("human", "Reference sentence: \n\n {sentences} \n\n Input sentence: {question}")]
)

## Multi-Query

In [14]:
def predict_label_with_multiquery_rag(statement: str, idx: int, total: int, max_docs: int = 5):

    queries = [statement]
    
    formatted_prompt = multiquery_prompt.format(question=statement)
    generated_queries = llm.invoke(formatted_prompt).content.strip().split("\n")
    queries.extend(generated_queries)
    
    all_retrieved_docs = []
    
    for query in queries:
        retrieved_docs = retriever_from_llm.invoke(query)[:max_docs]
        all_retrieved_docs.extend(retrieved_docs)
        
    unique_docs = list({doc.page_content: doc for doc in all_retrieved_docs}.values())

    retrieved_texts_str = "\n".join([doc.page_content for doc in unique_docs])
    
    formatted_prompt = classification_prompt.format(question=statement, sentences=retrieved_texts_str)
    
    prediction = structured_llm_labeler.invoke(formatted_prompt)
    
    return prediction

## Prediction

In [ ]:
results = []

for idx, row in tqdm(test.iterrows(), total=len(test), desc="Processing"):
    question = row["문장"]
    answer = row["비도덕여부"]
    
    prediction = predict_label_with_multiquery_rag(question, idx+1, len(test))

    results.append(
        {"question": question,
         "answer": answer,
         "prediction": prediction.prediction,
        }
    )

df_results = pd.DataFrame(results)

## Evaluation

In [ ]:
y_true = df_results["answer"].map({True: 1, False: 0})  
y_pred = df_results["prediction"].map({"True": 1, "False": 0})  

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1_final = f1_score(y_true, y_pred)
f1_macro = f1_score(y_true, y_pred, average="macro")
f1_weighted = f1_score(y_true, y_pred, average="weighted")

conf_matrix = confusion_matrix(y_true, y_pred)

print("\n===== Classification Performance Results =====")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_final:.4f}")
print(f"F1-score (Macro): {f1_macro:.4f}")
print(f"F1-score (Weighted): {f1_weighted:.4f}")

print("\n===== Classification Confusion Matrix =====")
print(conf_matrix)

print("\n===== Detailed Classification Report =====")
print(classification_report(y_true, y_pred))